# Scraping articles 
source: https://femminicidioitalia.info/lista/2022


In [12]:
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time 
import os 
import wget 
import getpass

In [31]:
s= Service('\Applications\chromedriver')

How to set up selenium : https://www.youtube.com/watch?v=95OqqfGi2Eg

In [36]:
#Optional- set options
chromeOptions = Options()
chromeOptions.headless= False 

#initialize driver 

driver= webdriver.Chrome(service=s, options= chromeOptions)

/var/folders/27/q_rl13_54jg1vgws9ntkdmbr0000gn/T/ipykernel_24294/3033477291.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chromeOptions.headless= False


In [39]:
def open_website(): 
    driver.get("https://femminicidioitalia.info/")
    print("opened")
    #cookie banner 
    cookie1=driver.find_element(By.CLASS_NAME, "fc-button fc-cta-consent fc-primary-button")
    time.sleep(5)
    cookie1.click()

open_website()
    

opened


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".fc-button fc-cta-consent fc-primary-button"}
  (Session info: chrome=109.0.5414.87)
Stacktrace:
0   chromedriver                        0x0000000104564fa8 chromedriver + 4886440
1   chromedriver                        0x00000001044e2643 chromedriver + 4351555
2   chromedriver                        0x0000000104130b27 chromedriver + 477991
3   chromedriver                        0x000000010417576c chromedriver + 759660
4   chromedriver                        0x00000001041759c1 chromedriver + 760257
5   chromedriver                        0x00000001041b9204 chromedriver + 1036804
6   chromedriver                        0x000000010419b5fd chromedriver + 914941
7   chromedriver                        0x00000001041b65fe chromedriver + 1025534
8   chromedriver                        0x000000010419b3a3 chromedriver + 914339
9   chromedriver                        0x000000010416557